# `juice_vector` setup instructions

## 1. `pg_vector` connection

#### 1.1. Create `config/pgvec_local.yaml` with the following content:

```yaml
dbname: juice_dev
user: Andrew_rw
password: ********
host: localhost
port: 5432
```


In [ ]:
import pandas as pd
import psycopg2
import yaml
import os

db_name = os.environ["DB_NAME"]
db_user = os.environ["DB_USER"] 
db_password = os.environ["DB_PASSWORD"] 
db_host = os.environ["DB_HOST"]

def lazy_pg_conn():
    return psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=5432,
    )


def lazy_pg_string():
    return f'postgresql+psycopg://{db_user}:{db_password}@{db_host}:5432/{db_name}'


def lazy_pg_credentials():
    config_string = {
        "dbname": db_name,
        "user": db_user,
        "password": db_password,
        "host": db_host,
        "port": 5432,
    }

    return config_string

In [ ]:

PG_CONNECTION_STRING = lazy_pg_conn(lazy_pg_credentials())


print(
    ">>> PG MASTER_CONNECTION {}".format(PG_CONNECTION_STRING).replace(
        "{}".format(PG_CONNECTION_STRING["password"]), "xxxxxx"
    )
)

FileNotFoundError: [Errno 2] No such file or directory: 'config/pgvec_g4a_m1.yaml'

#### 1.2. Install `pg_vector` extension

In [ ]:
# execute query with result fetch
def lazy_pg_query(config=None, query=None):

    conn = lazy_pg_conn(config)

    if conn is not None:
        try:
            with conn:
                with conn.cursor() as cur:
                    cur.execute(query)
                    result = cur.fetchall()
                    df = pd.DataFrame(
                        result, columns=[desc[0] for desc in cur.description]
                    )
                    return df
        except Exception as e:
            print("An error occurred:", e)
        conn.close()
    return None


# execute the query (no fetch)
def lazy_pg_query_nofetch(config=None, query=None):

    conn = lazy_pg_conn(config)

    if conn is not None:
        try:
            with conn:
                with conn.cursor() as cur:
                    cur.execute(query)
        except Exception as e:
            print("An error occurred:", e)
        conn.close()

In [ ]:
lazy_pg_query_nofetch(
    config=PG_CONNECTION_STRING, query="CREATE EXTENSION IF NOT EXISTS pg_trgm;"
)

lazy_pg_query(
    config=PG_CONNECTION_STRING,
    query="SELECT * FROM pg_extension WHERE extname = 'pg_trgm';",
)

In [ ]:
# lazy_pg_vector_setup(postgres_config=PG_CONFIG)

lazy_pg_query_nofetch(
    config=PG_CONNECTION_STRING, query="CREATE EXTENSION IF NOT EXISTS pg_trgm;"
)

lazy_pg_query(
    config=PG_CONNECTION_STRING,
    query="SELECT * FROM pg_extension WHERE extname = 'pg_trgm';",
)

should show something like this:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>oid</th>
      <th>extname</th>
      <th>extowner</th>
      <th>extnamespace</th>
      <th>extrelocatable</th>
      <th>extversion</th>
      <th>extconfig</th>
      <th>extcondition</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>170312</td>
      <td>pg_trgm</td>
      <td>16964</td>
      <td>2200</td>
      <td>True</td>
      <td>1.6</td>
      <td>None</td>
      <td>None</td>
    </tr>
  </tbody>
</table>
</div>

## 2. `postgreSQL` creates

#### 2.1. Create three views comnining `subbets` `truth_team` `player` `affiliation` and `games` tables

----

##### `games__ext` view:

```sql
    SELECT s.id, 
        s.description, 
        s.sport_affiliation_id, a."name" as affiliation_name, 
        s.subbet_type, s.duration, s.metric, 
        s.truth_team_id, tt1.display_name as "truth_team",
        s.opp_truth_team_id, tt2.display_name as "opp_truth_team",
        s.player_id, p1.first_name || ' ' || p1.last_name as player_name,
        s.opp_player_id, p2.first_name || ' ' || p2.last_name as opp_player_name,
        s.odds_jam_redis_odds_key,
        s."position", s.value, s.vig,
        s.game_id, 
        g.home_team_truth_id,th."name" as home_team_name, th.display_name as home_display_name, th.name_abbv_short as home_name_abbv_short, th.name_abbv_med as home_name_abbv_med, 
        g.away_team_truth_id,ta."name" as away_team_name, ta.display_name as away_display_name, ta.name_abbv_short as away_name_abbv_short, ta.name_abbv_med as away_name_abbv_med, 
        s.event_id, 
        s.is_standardized, 
        s.bet_id, s.result_status_id, s.start_date, s.created_at, s.updated_at
    FROM subbets s
        left join truth_teams tt1 on tt1.id = s.truth_team_id
        left join truth_teams tt2 on tt2.id = s.opp_truth_team_id
        left join players p1 on p1.id = s.player_id 
        left join players p2 on p2.id = s.opp_player_id 
        left JOIN games g on s.game_id = g.id
        left JOIN truth_teams th ON th.id = g.home_team_truth_id
        left JOIN truth_teams ta ON ta.id = g.away_team_truth_id 
        LEFT JOIN affiliations a ON s.sport_affiliation_id = a.id 
    where DATE(s.created_at) = '{date}'
        LIMIT 1000000;
```



----

##### `pta__ext` view:

```sql
    SELECT p.id as player_id, 
        t.id as team_id, 
        t.sport_affiliation_id, 
        p."position", p."number", 
        p.first_name || ' ' || p.last_name as player_name, 
        t."name" as team_name, 
        t.display_name, 
        t.name_abbv_short, 
        t.name_abbv_med, 
        a."name" as affiliation_name 
    FROM players p 
        LEFT JOIN truth_teams t ON p.team_id = t.id 
        LEFT JOIN affiliations a ON t.sport_affiliation_id = a.id 
    ORDER BY p.sport_affiliation_id, t.id, p.id asc 
        limit 100000;
```


----

#####  `games__ext` view:

```sql
    SELECT g.id, g.start_date, 
        g.sport_affiliation_id, g.home_team_truth_id,  
        th."name" as home_team_name, th.display_name as home_display_name, th.name_abbv_short as home_name_abbv_short, th.name_abbv_med as home_name_abbv_med, ah."name" as home_aff_name,
        g.away_team_truth_id,
        ta."name" as away_team_name, ta.display_name as away_display_name, ta.name_abbv_short as away_name_abbv_short, ta.name_abbv_med as away_name_abbv_med, aa."name" as away_aff_name,
        g.point_spread, g.over_under, g.home_team_implied_score, g.away_team_implied_score, g.home_team_moneyline, g.away_team_moneyline, 
        g.home_team_final_score, g.away_team_final_score, g.status, 
        g."createdAt", g."updatedAt"
    FROM games g
        left JOIN truth_teams th ON th.id = g.home_team_truth_id
        left JOIN truth_teams ta ON ta.id = g.away_team_truth_id 
        LEFT JOIN affiliations ah ON th.sport_affiliation_id = ah.id
        LEFT JOIN affiliations aa ON ta.sport_affiliation_id = aa.id
    where DATE(g.start_date) >= '{ date_min }' and DATE(g.start_date) <= '{ date_max }'
        limit 100000;
```
----


##### 2.2.3. `subbets_embed_`, `subbets_embed_queue`, `subbets_embed_queue_dist`, `subbets_embed_queue_dist_match`

----

```sql
CREATE VIEW public.subbets_embed_ AS
 SELECT id,
    document AS description,
    (cmetadata ->> 'subbet_type'::text) AS subbet_type,
    (cmetadata ->> 'metric'::text) AS metric,
    (cmetadata ->> 'duration'::text) AS duration,
    (cmetadata ->> 'position'::text) AS "position",
    (NULLIF((cmetadata ->> 'value'::text), ''::text))::double precision AS value,
    (NULLIF((cmetadata ->> 'vig'::text), ''::text))::double precision AS vig,
    (cmetadata ->> 'affiliation_name'::text) AS affiliation_name,
    (cmetadata ->> 'truth_team'::text) AS truth_team,
    (cmetadata ->> 'player_name'::text) AS player_name,
    (cmetadata ->> 'opp_truth_team'::text) AS opp_truth_team,
    (cmetadata ->> 'opp_player_name'::text) AS opp_player_name,
    (cmetadata ->> 'home_display_name'::text) AS home_display_name,
    (cmetadata ->> 'away_display_name'::text) AS away_display_name,
    (cmetadata ->> 'created_at'::text) AS created_at,
    (cmetadata ->> 'updated_at'::text) AS updated_at
   FROM public.langchain_pg_embedding__subbets__g4a lpe;
DROP VIEW public.subbets_embed_;

```
    


----

```sql


CREATE VIEW public.subbets_embed_queue AS
 SELECT id,
    description,
    subbet_type,
    metric,
    duration,
    "position",
    value,
    vig,
    affiliation_name,
    truth_team,
    player_name,
    opp_truth_team,
    opp_player_name,
    home_display_name,
    away_display_name,
    created_at,
    updated_at
   FROM public.subbets_embed_ se
  WHERE ((affiliation_name IS NULL) OR ((affiliation_name = 'Unknown'::text) AND (NOT ((id)::text IN ( SELECT sl.id
           FROM public.subbet_log sl)))));

DROP VIEW public.subbets_embed_queue;

```
    


----

```sql

CREATE VIEW public.subbets_embed_queue_dist AS
 SELECT seq.id,
    lpe.id AS id_match,
    (qe.embedding OPERATOR(public.<->) lpe.embedding) AS distance
   FROM ((public.subbets_embed_queue seq
     JOIN public.langchain_pg_embedding__subbets__g4a qe ON (((qe.id)::text = (seq.id)::text)))
     JOIN LATERAL ( SELECT langchain_pg_embedding__subbets__g4a.id,
            langchain_pg_embedding__subbets__g4a.embedding
           FROM public.langchain_pg_embedding__subbets__g4a
          WHERE (NOT ((langchain_pg_embedding__subbets__g4a.id)::text IN ( SELECT subbets_embed_queue.id
                   FROM public.subbets_embed_queue)))
          ORDER BY (qe.embedding OPERATOR(public.<->) langchain_pg_embedding__subbets__g4a.embedding)
         LIMIT 1) lpe ON (true));
DROP VIEW public.subbets_embed_queue_dist;
```


----

```sql

CREATE VIEW public.subbets_embed_queue_dist_match AS
 SELECT lpe1.id,
    lpe1.document AS description,
    seqd.distance,
    lpe2.id AS id_match,
    lpe2.document AS match_description,
    (lpe2.cmetadata ->> 'subbet_type'::text) AS subbet_type,
    (lpe2.cmetadata ->> 'metric'::text) AS metric,
    (lpe2.cmetadata ->> 'duration'::text) AS duration,
    (lpe2.cmetadata ->> 'position'::text) AS "position",
    (lpe2.cmetadata ->> 'value'::text) AS value,
    (lpe1.cmetadata ->> 'vig'::text) AS vig,
    (lpe2.cmetadata ->> 'affiliation_name'::text) AS affiliation_name,
    (lpe2.cmetadata ->> 'truth_team'::text) AS truth_team,
    (lpe2.cmetadata ->> 'player_name'::text) AS player_name,
    (lpe2.cmetadata ->> 'opp_truth_team'::text) AS opp_truth_team,
    (lpe2.cmetadata ->> 'opp_player_name'::text) AS opp_player_name,
    (lpe2.cmetadata ->> 'home_display_name'::text) AS home_display_name,
    (lpe2.cmetadata ->> 'away_display_name'::text) AS away_display_name
   FROM ((public.subbets_embed_queue_dist seqd
     JOIN ( SELECT langchain_pg_embedding__subbets__g4a.id,
            langchain_pg_embedding__subbets__g4a.document,
            langchain_pg_embedding__subbets__g4a.cmetadata
           FROM public.langchain_pg_embedding__subbets__g4a) lpe1 ON (((lpe1.id)::text = (seqd.id)::text)))
     JOIN ( SELECT langchain_pg_embedding__subbets__g4a.id,
            langchain_pg_embedding__subbets__g4a.document,
            langchain_pg_embedding__subbets__g4a.cmetadata
           FROM public.langchain_pg_embedding__subbets__g4a) lpe2 ON (((lpe2.id)::text = (seqd.id_match)::text)));

DROP VIEW public.subbets_embed_queue_dist_match;

```


##### 2.2.4. `subbets_truth_`, `subbets_truth_bool`, `subbets_truth_bool_sums` and `subbet_log`,



----
```sql
CREATE VIEW public.subbets_truth_ AS
 SELECT id,
    description,
    subbet_type,
    metric,
    duration,
    "position",
    value,
    vig,
    affiliation_name,
    truth_team,
    player_name,
    opp_truth_team,
    opp_player_name,
    home_display_name,
    away_display_name,
    created_at,
    updated_at
   FROM public.subbets_ext se;

DROP VIEW public.subbets_truth_;

```

----

```sql
CREATE VIEW public.subbets_truth_bool AS
 SELECT st.id,
    (NOT (st.description IS DISTINCT FROM (lpe.cmetadata ->> 'description'::text))) AS description,
    (NOT (st.subbet_type IS DISTINCT FROM (lpe.cmetadata ->> 'subbet_type'::text))) AS subbet_type,
    (NOT (st.metric IS DISTINCT FROM (lpe.cmetadata ->> 'metric'::text))) AS metric,
    (NOT (st.duration IS DISTINCT FROM (lpe.cmetadata ->> 'duration'::text))) AS duration,
    (NOT (st."position" IS DISTINCT FROM (lpe.cmetadata ->> 'position'::text))) AS "position",
    (NOT ((st.value)::text IS DISTINCT FROM (lpe.cmetadata ->> 'value'::text))) AS value,
    (NOT ((st.vig)::text IS DISTINCT FROM (lpe.cmetadata ->> 'vig'::text))) AS vig,
    (NOT (st.affiliation_name IS DISTINCT FROM (lpe.cmetadata ->> 'affiliation_name'::text))) AS affiliation_name,
    (NOT (st.truth_team IS DISTINCT FROM (lpe.cmetadata ->> 'truth_team'::text))) AS truth_team,
    (NOT (st.player_name IS DISTINCT FROM (lpe.cmetadata ->> 'player_name'::text))) AS player_name,
    (NOT (st.opp_truth_team IS DISTINCT FROM (lpe.cmetadata ->> 'opp_truth_team'::text))) AS opp_truth_team,
    (NOT ((st.opp_player_name)::text IS DISTINCT FROM (lpe.cmetadata ->> 'opp_player_name'::text))) AS opp_player_name,
    (NOT (st.home_display_name IS DISTINCT FROM (lpe.cmetadata ->> 'home_display_name'::text))) AS home_display_name,
    (NOT (st.away_display_name IS DISTINCT FROM (lpe.cmetadata ->> 'away_display_name'::text))) AS away_display_name,
    (NOT (st.created_at IS DISTINCT FROM (lpe.cmetadata ->> 'created_at'::text))) AS created_at,
    (NOT (st.updated_at IS DISTINCT FROM (lpe.cmetadata ->> 'udpated_at'::text))) AS updated_at
   FROM (public.subbets_truth_ st
     JOIN public.langchain_pg_embedding lpe ON (((lpe.id)::text = (st.id)::text)))
  WHERE (st.updated_at < (lpe.cmetadata ->> 'updated_at'::text));

DROP VIEW public.subbets_truth_bool;

```



----

```sql
CREATE VIEW public.subbets_truth_bool_sums AS
 SELECT se.affiliation_name AS group_name,
    count(stb.id) AS total_count,
    sum(
        CASE
            WHEN stb.description THEN 1
            ELSE 0
        END) AS description,
    sum(
        CASE
            WHEN stb.subbet_type THEN 1
            ELSE 0
        END) AS subbet_type,
    sum(
        CASE
            WHEN stb.metric THEN 1
            ELSE 0
        END) AS metric,
    sum(
        CASE
            WHEN stb.duration THEN 1
            ELSE 0
        END) AS duration,
    sum(
        CASE
            WHEN stb."position" THEN 1
            ELSE 0
        END) AS "position",
    sum(
        CASE
            WHEN stb.value THEN 1
            ELSE 0
        END) AS value,
    sum(
        CASE
            WHEN stb.vig THEN 1
            ELSE 0
        END) AS vig,
    sum(
        CASE
            WHEN stb.affiliation_name THEN 1
            ELSE 0
        END) AS affiliation_name,
    sum(
        CASE
            WHEN stb.truth_team THEN 1
            ELSE 0
        END) AS truth_team,
    sum(
        CASE
            WHEN stb.player_name THEN 1
            ELSE 0
        END) AS player_name,
    sum(
        CASE
            WHEN stb.opp_truth_team THEN 1
            ELSE 0
        END) AS opp_truth_team,
    sum(
        CASE
            WHEN stb.opp_player_name THEN 1
            ELSE 0
        END) AS opp_player_name,
    sum(
        CASE
            WHEN stb.home_display_name THEN 1
            ELSE 0
        END) AS home_display_name,
    sum(
        CASE
            WHEN stb.away_display_name THEN 1
            ELSE 0
        END) AS away_display_name,
    sum(
        CASE
            WHEN stb.created_at THEN 1
            ELSE 0
        END) AS created_at,
    sum(
        CASE
            WHEN stb.updated_at THEN 1
            ELSE 0
        END) AS updated_at
   FROM (public.subbets_truth_bool stb
     JOIN public.subbets_embed_ se ON (((se.id)::text = (stb.id)::text)))
  GROUP BY se.affiliation_name
  ORDER BY (count(stb.id)) DESC;
```
----



### 2.2. See `juice_vector_schema.sql`

```zsh
pg_dump: creating FUNCTION "public.subbet_metadata_10g()" 
pg_dump: creating FUNCTION "public.subbets_ext_delete_existing_id()" 
pg_dump: creating TABLE "public.foreign_subbets"

pg_dump: creating TABLE "public.langchain_pg_collection" 
pg_dump: creating TABLE "public.langchain_pg_embedding"

pg_dump: creating VIEW "public.langchain_pg_embedding_gms_g4a" 
pg_dump: creating VIEW "public.langchain_pg_embedding_pta_g4a" 
g_dump: creating VIEW "public.langchain_pg_embedding__subbets_g4a" 

pg_dump: creating VIEW "public.players_embed_pta_" 
pg_dump: creating VIEW "public.teams_embed_pta_"

pg_dump: creating TABLE "public.games_ext"
pg_dump: creating TABLE "public.players_ext" 
pg_dump: creating TABLE "public.subbets_ext" 
pg_dump: creating TABLE "public.subbet_log" 

pg_dump: creating VIEW "public.subbets_embed_" 
pg_dump: creating VIEW "public.subbets_embed_queue" 
pg_dump: creating VIEW "public.subbets_embed_queue_dist" 
pg_dump: creating VIEW "public.subbets_embed_queue_dist_match" 

pg_dump: creating VIEW "public.subbets_queue" 
pg_dump: creating VIEW "public.subbets_truth_"
pg_dump: creating VIEW "public.subbets_truth_bool" 
pg_dump: creating VIEW "public.subbets_truth_bool_sums" 

pg_dump: creating VIEW "public.subbets_truth_players" 
pg_dump: creating VIEW "public.subbets_truth_teams"

```

## 3. `ollama` inference setup

### 3.1. Install `ollama` [see platform instructions here](https://github.com/ollama/ollama)

#### 3.2. Download some `models`

```bash
    ollama pull llama3:latest
```



## 4. `juice_vector` Containerization

### 4.1. Docker?

```bash
    docker pull juice_docker:latest
    docker run -d -p 5001:5001 juice_docker:latest
```

### 4.2. Install python dependencies


##### `Langchain` for tooling
```bash
    python3.11 -m pip install -U langchain
```

##### `Ollama` for inference
```bash
    python3.11 -m pip install -U ollama
```

##### `GPT4All` for embeddings
```bash
    python3.11 -m pip install -U gpt4all
```
